Reformatting Data to work w/ JS web app

In [1]:
import pandas as pd
import os 
import glob

In [2]:
messy_filepath = r'\\wsl.localhost\Ubuntu\home\garrett\CLICK\clickdata\messydata\click-bc.appspot.com'
clean_filepath = r'\\wsl.localhost\Ubuntu\home\garrett\CLICK\clickdata\public\data-storage\telem'
clean_filepath_fpga = r'\\wsl.localhost\Ubuntu\home\garrett\CLICK\clickdata\public\data-storage\fpga'

In [3]:
all_folders = []
for flyover_data in glob.glob(messy_filepath + '\*'):
    folder_name = flyover_data.split('\\')[-1]
    all_folders.append(folder_name)
print(len(all_folders), all_folders)

442 ['9fWnDVpMzec0pcGlf4Mv', '6JUxjzf5w2bMlyD1FqL6', '8AIS40VT1Izj9eTvcPru', 'lHuzpnYlvtLHFz2LjHrd', 'G52igW51DNhUBBuVicF2', 'q2Rpr9lYVPTEcZ1dwfZa', 'HGHyoAdmQjrBPN3zixKy', '8b9JqawzrjKwITEuCWrg', 'agnNPCSKczcPTO1OhGK8', 'ny9DoNOaJ3ZF9ztZoFaR', 'q6clRyBs59K39hnHuYH3', 'DPUBZv8InzHlWJYNWItt', '17c5WEnbA5Bhjjbxr7dJ', 'OkCcLOfCnAAzoNpiMQFl', 'j5u66x3Wt5h8OgKf2M7l', 'aPvBOoUVm7HfbUQEaVOc', 'MU64D54tPIBIrHwiBWYL', 'mQbMxWqEDV7fwuPOIr0m', 'fGmluKixCgPHhxQPga1g', 'QS7b44rXDTHrHv7Rfrrg', 'V9rvFjGOW9ELsN3mr5hk', 'ho3KMsCBq7Ohll0eetWO', 'T7F1vUo25e8VMg4Blcm4', 'xvtfCHBVI7lXZjyefTDG', 'YEHoHniGgdLJrjkyEN1L', 'HpLcyZ0HnKnBfbhL2sun', 'IIr6LyfZPBzbUAJ48wNd', 'NNtue688UWx5HkmpMwmG', 'IkXoU8ehIZfIOjM1DRJW', 'N5D3Y2VsmSdbbwhAJe5R', 'oR2Wnoj9GhwKKpOpDwmk', 'wLmtLiQ25lAvdLiZFx4a', 'OLBaTIJMggdp9wLRwdVX', 'Dy9z9dRmqiMU2Gu8paeM', '9AdJrQaBaHqSnfC93D24', 'CPJnrjpLdMHWhveRveng', 'Xc1FwnDfeO2acsi8dhuA', 'ARjjidrnrzp1N0GMhtM1', 'vSNMTZllDSZJCJ7aM7az', 'TafkK82Uw5D12Am5Rd97', 'xZzzsYwE0fVhILAO3BUq', 'R8ZHqbM66T

In [5]:
valid_data = {}
fpga_data = {}
weird_errors = []
for folder in all_folders:
    flypath = glob.glob(messy_filepath + f'\\{folder}\\' + '*.csv')
    if flypath:
        try:
            df = pd.read_csv(flypath[0])

            if 'Unnamed 1:' in df.columns:
                for i in range(1, 6):
                    df = pd.read_csv(flypath[0], header=i)
                    if 'Unnamed 1:' not in df:
                        break
                if 'Unnamed 1:' in df:
                    raise KeyError
                    
            if 'HK_FPGA' in flypath[0]:
                if 'TIME' in df.columns:
                    timestamp = pd.to_datetime(df['TIME'].iloc[:1], format='%Y-%m-%d %H:%M:%S %z').dt.strftime('%Y-%m-%d_%H:%M:%S').iloc[0]
                elif 'time' in df.columns:
                    timestamp = pd.to_datetime(df['time'].iloc[:1], format='%Y-%m-%d %H:%M:%S %z').dt.strftime('%Y-%m-%d_%H:%M:%S').iloc[0]
                    
                fpga_data[flypath[0]] = timestamp
            else:
                
                if "UUT COMMAND_TLM_PACKET_TIMESECONDS" in df.columns:
                    timestamp = pd.to_datetime(df['UUT COMMAND_TLM_PACKET_TIMESECONDS'].iloc[:1], unit='s').dt.strftime('%Y-%m-%d_%H:%M:%S').iloc[0]

                elif "UUT_COMMAND_TLM_PACKET_TIMESECONDS" in df.columns:
                    timestamp = pd.to_datetime(df['UUT_COMMAND_TLM_PACKET_TIMESECONDS'].iloc[:1], unit='s').dt.strftime('%Y-%m-%d_%H:%M:%S').iloc[0]

                elif "PACKET_TIMESECONDS" in df.columns:
                    timestamp = pd.to_datetime(df['PACKET_TIMESECONDS'].iloc[:1], unit='s').dt.strftime('%Y-%m-%d_%H:%M:%S').iloc[0]
                    
                # if len(df) > 10:
                valid_data[flypath[0]] = timestamp
        except Exception as e:
            print(e)
            weird_errors.append(flypath[0])  # these files must be corrupted, can't decode them with invalid bytes. Only 3/374 tho
            
print(len(valid_data), len(fpga_data), len(weird_errors))  # 408, 4

C:\Users\gman_\AppData\Local\Temp\ipykernel_22588\3056947509.py:8: DtypeWarning: Columns (1,3,11,19,48,62,73,74,75,76,77,78,79,81,82,83,91,92,93,100,101,102,112,113) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(flypath[0])
C:\Users\gman_\AppData\Local\Temp\ipykernel_22588\3056947509.py:8: DtypeWarning: Columns (1,3,11,19,48,62,73,74,75,76,77,78,79,81,82,83,91,92,93,100,101,102,112,113) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(flypath[0])
C:\Users\gman_\AppData\Local\Temp\ipykernel_22588\3056947509.py:8: DtypeWarning: Columns (1,3,11,18,19,21,50,64,75,76,77,78,79,80,81,82,83,85,86,87,95,96,97,104,105,106,116,117) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(flypath[0])
C:\Users\gman_\AppData\Local\Temp\ipykernel_22588\3056947509.py:8: DtypeWarning: Columns (1,3,11,19,48,62,73,74,75,76,77,78,79,81,82,83,91,92,93,100,101,102,112,113) have mixe

'utf-8' codec can't decode byte 0xd0 in position 0: invalid continuation byte


C:\Users\gman_\AppData\Local\Temp\ipykernel_22588\3056947509.py:8: DtypeWarning: Columns (1,3,11,19,48,62,73,74,75,76,77,78,79,81,82,83,91,92,93,100,101,102,112,113) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(flypath[0])
C:\Users\gman_\AppData\Local\Temp\ipykernel_22588\3056947509.py:8: DtypeWarning: Columns (1,3,11,19,48,62,73,74,75,76,77,78,79,81,82,83,91,92,93,100,101,102,112,113) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(flypath[0])
C:\Users\gman_\AppData\Local\Temp\ipykernel_22588\3056947509.py:8: DtypeWarning: Columns (1,3,11,19,48,62,73,74,75,76,77,78,79,81,82,83,91,92,93,100,101,102,112,113) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(flypath[0])
C:\Users\gman_\AppData\Local\Temp\ipykernel_22588\3056947509.py:8: DtypeWarning: Columns (1,3,11,19,48,62,73,74,75,76,77,78,79,81,82,83,91,92,93,100,101,102,112,113) have mixed types. Spe

'utf-8' codec can't decode byte 0xd0 in position 0: invalid continuation byte


C:\Users\gman_\AppData\Local\Temp\ipykernel_22588\3056947509.py:8: DtypeWarning: Columns (1,3,11,19,48,62,73,74,75,76,77,78,79,81,82,83,91,92,93,100,101,102,112,113) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(flypath[0])
C:\Users\gman_\AppData\Local\Temp\ipykernel_22588\3056947509.py:8: DtypeWarning: Columns (1,3,11,18,19,21,50,64,75,76,77,78,79,80,81,82,83,85,86,87,95,96,97,104,105,106,116,117) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(flypath[0])
C:\Users\gman_\AppData\Local\Temp\ipykernel_22588\3056947509.py:8: DtypeWarning: Columns (1,3,11,19,48,62,73,74,75,76,77,78,79,81,82,83,91,92,93,100,101,102,112,113) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(flypath[0])
C:\Users\gman_\AppData\Local\Temp\ipykernel_22588\3056947509.py:8: DtypeWarning: Columns (1,3,11,18,19,21,50,64,75,76,77,78,79,80,81,82,83,85,86,87,95,96,97,104,105,106,116,11

'utf-8' codec can't decode byte 0xd0 in position 0: invalid continuation byte


C:\Users\gman_\AppData\Local\Temp\ipykernel_22588\3056947509.py:8: DtypeWarning: Columns (1,3,11,19,48,62,73,74,75,76,77,78,79,81,82,83,91,92,93,100,101,102,112,113) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(flypath[0])
C:\Users\gman_\AppData\Local\Temp\ipykernel_22588\3056947509.py:8: DtypeWarning: Columns (1,3,11,18,19,21,50,64,75,76,77,78,79,80,81,82,83,85,86,87,95,96,97,104,105,106,116,117) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(flypath[0])


315 94 3


C:\Users\gman_\AppData\Local\Temp\ipykernel_22588\3056947509.py:8: DtypeWarning: Columns (1,3,11,19,48,62,73,74,75,76,77,78,79,81,82,83,91,92,93,100,101,102,112,113) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(flypath[0])


In [10]:
for vpath in valid_data:
    df = pd.read_csv(vpath)
    
    if 'Unnamed 1:' in df.columns:
        for i in range(1, 6):
            df = pd.read_csv(vpath, header=i)
            if 'Unnamed 1:' not in df:
                break
    
    if "UUT COMMAND_TLM_PACKET_TIMESECONDS" in df.columns:
        df['timestamp_'] = pd.to_datetime(df['UUT COMMAND_TLM_PACKET_TIMESECONDS'], unit='s')

    elif "UUT_COMMAND_TLM_PACKET_TIMESECONDS" in df.columns:
        df['timestamp_'] = pd.to_datetime(df['UUT_COMMAND_TLM_PACKET_TIMESECONDS'], unit='s')

    elif "PACKET_TIMESECONDS" in df.columns:
        df['timestamp_'] = pd.to_datetime(df['PACKET_TIMESECONDS'], unit='s')

    df.to_csv(clean_filepath + '\\' + valid_data[vpath].replace(':', '-') + '.csv')


for fpath in fpga_data:
    
    df = pd.read_csv(fpath)
    
    if 'Unnamed 1:' in df.columns:
        for i in range(1, 6):
            df = pd.read_csv(fpath, header=i)
            if 'Unnamed 1:' not in df:
                break

    if 'time' in df.columns:
        df['timestamp_'] = pd.to_datetime(df['time'].iloc[:1], format='%Y-%m-%d %H:%M:%S %z')
    elif 'TIME' in df.columns:
        df['timestamp_'] = pd.to_datetime(df['TIME'].iloc[:1], format='%Y-%m-%d %H:%M:%S %z')

    df.to_csv(clean_filepath_fpga + '\\' + fpga_data[fpath].replace(':', '-') + '.csv')


C:\Users\gman_\AppData\Local\Temp\ipykernel_22588\682668799.py:2: DtypeWarning: Columns (1,3,11,19,48,62,73,74,75,76,77,78,79,81,82,83,91,92,93,100,101,102,112,113) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(vpath)
C:\Users\gman_\AppData\Local\Temp\ipykernel_22588\682668799.py:2: DtypeWarning: Columns (1,3,11,19,48,62,73,74,75,76,77,78,79,81,82,83,91,92,93,100,101,102,112,113) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(vpath)
C:\Users\gman_\AppData\Local\Temp\ipykernel_22588\682668799.py:2: DtypeWarning: Columns (1,3,11,18,19,21,50,64,75,76,77,78,79,80,81,82,83,85,86,87,95,96,97,104,105,106,116,117) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(vpath)
C:\Users\gman_\AppData\Local\Temp\ipykernel_22588\682668799.py:2: DtypeWarning: Columns (1,3,11,19,48,62,73,74,75,76,77,78,79,81,82,83,91,92,93,100,101,102,112,113) have mixed types. Specify dt

In [5]:
valid_data

{'\\\\wsl.localhost\\Ubuntu\\home\\garrett\\CLICK\\clickdata\\messydata\\click-bc.appspot.com\\6JUxjzf5w2bMlyD1FqL6\\2023_03_27_09_55_28_tlm.csv': '2023-03-27_13:55:32',
 '\\\\wsl.localhost\\Ubuntu\\home\\garrett\\CLICK\\clickdata\\messydata\\click-bc.appspot.com\\lHuzpnYlvtLHFz2LjHrd\\2023_02_02_14_04_42_tlm.csv': '2023-02-02_19:04:42',
 '\\\\wsl.localhost\\Ubuntu\\home\\garrett\\CLICK\\clickdata\\messydata\\click-bc.appspot.com\\G52igW51DNhUBBuVicF2\\2022_09_13_10_43_45_tlm.csv': '2023-02-02_19:04:42',
 '\\\\wsl.localhost\\Ubuntu\\home\\garrett\\CLICK\\clickdata\\messydata\\click-bc.appspot.com\\q2Rpr9lYVPTEcZ1dwfZa\\2023_03_06_14_34_49_tlm.csv': '2023-03-06_19:34:49',
 '\\\\wsl.localhost\\Ubuntu\\home\\garrett\\CLICK\\clickdata\\messydata\\click-bc.appspot.com\\HGHyoAdmQjrBPN3zixKy\\HK_FPGA_2021-11-02_13-58-34.csv': '2023-03-06_19:34:49',
 '\\\\wsl.localhost\\Ubuntu\\home\\garrett\\CLICK\\clickdata\\messydata\\click-bc.appspot.com\\8b9JqawzrjKwITEuCWrg\\2022_10_25_17_03_46_tlm.csv':

In [8]:
fpga = pd.read_csv( '\\\\wsl.localhost\\Ubuntu\\home\\garrett\\CLICK\\clickdata\\messydata\\click-bc.appspot.com\\fZpBaNF3BkLUwZGYn1Ec\\HK_FPGA_2021-11-02_13-58-34X.csv')
fpga

,TIME,APID_VALID,CRC_VALID,FPGA_COUNTER,0,1,2,3,4,32,...,610,611,DAC_1_A,DAC_1_B,DAC_1_C,DAC_1_D,DAC_2_A,DAC_2_B,DAC_2_C,DAC_2_D
0,2021-11-02 13:59:25 -0600,1,1,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
1,2021-11-02 13:59:30 -0600,1,1,1,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
2,2021-11-02 13:59:35 -0600,1,1,2,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
3,2021-11-02 13:59:40 -0600,1,1,3,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
4,2021-11-02 13:59:45 -0600,1,1,4,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2021-11-02 14:19:41 -0600,1,1,93,0,0,0,0,0,15,...,621,34.0,0,0,0,0,0,0,0,0
236,2021-11-02 14:19:46 -0600,1,1,94,0,0,0,0,0,15,...,621,34.0,0,0,0,0,0,0,0,0
237,2021-11-02 14:19:51 -0600,1,1,95,0,0,0,0,0,15,...,621,34.0,0,0,0,0,0,0,0,0
238,2021-11-02 14:19:56 -0600,1,1,96,0,0,0,0,0,15,...,621,34.0,0,0,0,0,0,0,0,0
